In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow import keras

import pickle

pickle_in = open("X_image.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_image.pickle","rb")
y = pickle.load(pickle_in)

X = X.astype("float32")/255.0

class MLPHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        
        model.add(Flatten())
        
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        model.add(
            Dense(
                units=hp.Int(
                    'units_2',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_2',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        model.add(
            Dense(
                units=hp.Int(
                    'units_3',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_3',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        model.add(
            Dense(
                units=hp.Int(
                    'units_4',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_4',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_4',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
            )),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
NUM_CLASSES = 3  # happy neutral sad number of classes
INPUT_SHAPE = (48, 48, 1) 
SEED = 1
HYPERBAND_MAX_EPOCHS = 15
MAX_TRIALS = 25
EXECUTION_PER_TRIAL = 2

hypermodel = MLPHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='MLP8'
)

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='MLP8'
)

N_EPOCH_SEARCH = 15
BATCH_SIZE = 8

tuner.search(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH_SEARCH, validation_split=0.2)

tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Trial 30 Complete [00h 00m 29s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.9000000059604645
Total elapsed time: 00h 08m 18s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hyperband\MLP8
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 352
dense_activation: sigmoid
dropout: 0.05
units_2: 192
dense_activation_2: relu
dropout_2: 0.05
units_3: 384
dense_activation_3: relu
dropout_3: 0.2
units_4: 256
dense_activation_4: relu
dropout_4: 0.25
learning_rate: 0.00036151371126048177
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 123c3cbd230a2e6f294d5828efd10f66
Score: 0.9000000059604645
Trial summary
Hyperparameters:
units: 448
dense_activation: tanh
dropout: 0.2
units_2: 256
dense_activation_2: sigmoid
dropout_2: 0.25
units_3: 224
dense_activation_3: relu
dropout_3: 0.25
units_4: 192
dense_activation_4: sigmoid
dropout_4: 0.0
learning_rate: 0.00026384087338560336
tuner/epoc